# Aspect Extraction

In [22]:
from pprint import pprint # pretty print


In [23]:
# We get started by importing spacy
import spacy
nlp = spacy.load("en_core_web_lg")

In [24]:
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x1ceb0e0d6d0>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x1ceb0e0f470>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1ceb0d80e40>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x1ceb0f81b90>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x1ceaf792290>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1ceaf8cee30>)]

In [25]:
sentences = [
  'The food we had yesterday was delicious',
  'My time in Italy was very enjoyable',
  'I found the meal to be tasty',
  'The internet was slow.',
  'Our experience was suboptimal',
  'anc work in the first 3 days but the proximity sensor on the left bud stopped working',
  'shipped out immediately, received the item in good condition, no damage well packed, thank you shopee and thank you seller, will order again 👍😁👌🫰',
  "Tysm for delivering my items safely. I only expected the seller gave some freebies, but he/she didn't 😂😂😾",
  'Sobrang tagal lang dumating. Pero okay lang, sulit naman. Nakuha ko sya sa halagang 999 pesos. Nagsale kasi. 😍 Thank youuu! Legit !',
  "been using this buds for 2 weeks na and I notice that sometimes it took so long to connect on my laptop and phone I don't know why. then sometimes nagkaproblem sound nya parang basag yung tunog",
  'wrong color. poor packaging',
  'Delivery and shipment were both okay.',
  """If you intend to use this to attract your man, be prepared for a ravenous, passionate result. Do not use if you don’t intend to follow through with your intended target. That’s a d*** move. Plus this stuff is potent and you have no idea how a man might respond to its siren call. It’s basically a love potion so handle with care. But it absolutely works!
Tip 1: let it settle after application, it’s strong but nice smelling at first, but mellows out over time.
Tip 2: combine with your perfume of choice, particularly one that has a positive olfactory memory trigger with your partner/intended target.
Tip 3: don’t use too much, or it becomes a turn OFF (tested and confirmed)! A dab or small swipe on wrists and chest or behind ear/jaw should do it
Worth the money. Have fun!""",
  "I was hesitant at first but I heard lots of good things about this brand so I got it since I’ve been kinda crushing on a friend of mine and he was coming over for the weekend. I put it on and he complimented the way I smelled twice and by the end of the night he confessed to me and were dating. Also a weird but cute side effect is that my cats and dog were very lovey dovey on me too. It worked wonders for me. Buy this IMMEDIATELY"
]

In [26]:
from spacy import displacy
testtext = nlp(sentences[-1])
displacy.render(testtext, style = "dep")

In [27]:
displacy.render(testtext, style = "ent")

In [28]:
displacy.render(testtext, style='dep', options={'compact': True})

For each token inside our sentences, we can see the dependency thanks to spacy’s dependency parsing and the POS (Part-Of-Speech) tags. We’re also paying attention to the child tokens, so that we’re able to pick up intensifiers such as “very”, “quite”, and more.

In [29]:
for sentence in sentences:
  doc = nlp(sentence)
  for token in doc:
    print(token.text, spacy.explain(token.dep_), token.head.text, spacy.explain(token.head.pos_),
      token.pos_,[child for child in token.children], sep=' | ')

The | determiner | food | noun | DET | []
food | nominal subject | was | auxiliary | NOUN | [The, had]
we | nominal subject | had | verb | PRON | []
had | relative clause modifier | food | noun | VERB | [we, yesterday]
yesterday | noun phrase as adverbial modifier | had | verb | NOUN | []
was | root | was | auxiliary | AUX | [food, delicious]
delicious | adjectival complement | was | auxiliary | ADJ | []
My | possession modifier | time | noun | PRON | []
time | nominal subject | was | auxiliary | NOUN | [My, in]
in | prepositional modifier | time | noun | ADP | [Italy]
Italy | object of preposition | in | adposition | PROPN | []
was | root | was | auxiliary | AUX | [time, enjoyable]
very | adverbial modifier | enjoyable | adjective | ADV | []
enjoyable | adjectival complement | was | auxiliary | ADJ | [very]
I | nominal subject | found | verb | PRON | []
found | root | found | verb | VERB | [I, be]
the | determiner | meal | noun | DET | []
meal | nominal subject | be | auxiliary | NOUN

Let’s see how to pick up the sentiment descriptions first.

In [30]:
for sentence in sentences:
  doc = nlp(sentence)
  descriptive_term = ''
  for token in doc:
    if token.pos_ == 'ADJ':
      descriptive_term = token
  print(sentence)
  print(f'>{descriptive_term}')

The food we had yesterday was delicious
>delicious
My time in Italy was very enjoyable
>enjoyable
I found the meal to be tasty
>tasty
The internet was slow.
>slow
Our experience was suboptimal
>suboptimal
anc work in the first 3 days but the proximity sensor on the left bud stopped working
>left
shipped out immediately, received the item in good condition, no damage well packed, thank you shopee and thank you seller, will order again 👍😁👌🫰
>good
Tysm for delivering my items safely. I only expected the seller gave some freebies, but he/she didn't 😂😂😾
>
Sobrang tagal lang dumating. Pero okay lang, sulit naman. Nakuha ko sya sa halagang 999 pesos. Nagsale kasi. 😍 Thank youuu! Legit !
>Legit
been using this buds for 2 weeks na and I notice that sometimes it took so long to connect on my laptop and phone I don't know why. then sometimes nagkaproblem sound nya parang basag yung tunog
>long
wrong color. poor packaging
>poor
Delivery and shipment were both okay.
>okay
If you intend to use this 

In [31]:
for sentence in sentences:
  doc = nlp(sentence)
  descriptive_term = ''
  for token in doc:
    if token.pos_ == 'ADJ':
      prepend = ''
      for child in token.children:
        if child.pos_ != 'ADV':
          continue
        prepend += child.text + ' '
      descriptive_term = prepend + token.text
  print(sentence)
  print(f'>{descriptive_term}')

The food we had yesterday was delicious
>delicious
My time in Italy was very enjoyable
>very enjoyable
I found the meal to be tasty
>tasty
The internet was slow.
>slow
Our experience was suboptimal
>suboptimal
anc work in the first 3 days but the proximity sensor on the left bud stopped working
>left
shipped out immediately, received the item in good condition, no damage well packed, thank you shopee and thank you seller, will order again 👍😁👌🫰
>good
Tysm for delivering my items safely. I only expected the seller gave some freebies, but he/she didn't 😂😂😾
>
Sobrang tagal lang dumating. Pero okay lang, sulit naman. Nakuha ko sya sa halagang 999 pesos. Nagsale kasi. 😍 Thank youuu! Legit !
>Legit
been using this buds for 2 weeks na and I notice that sometimes it took so long to connect on my laptop and phone I don't know why. then sometimes nagkaproblem sound nya parang basag yung tunog
>so long
wrong color. poor packaging
>poor
Delivery and shipment were both okay.
>okay
If you intend to u

As you can see, this time around we picked up very enjoyable as well. Our simplistic algorithm is able to pick up adverbs. It checks for child tokens for each adjective and picks up the adverbs such as “very”, “quite”, etc.

In [32]:
displacy.render(nlp(sentences[5]), style='dep', options={'compact': True})

### CUSTOM RULES
[ASPECT SENTIMENT GITHUB LINK](https://github.com/ishikaarora/Aspect-Sentiment-Analysis-on-Amazon-Reviews/blob/master/team01final/code/src/models/aspect_extraction.py)

In [39]:
# apply_extraction FUNCTION: https://github.com/ishikaarora/Aspect-Sentiment-Analysis-on-Amazon-Reviews/blob/master/team01final/code/src/models/aspect_extraction.py
prod_pronouns = ['it','this','they','these']

def apply_extraction(reviews,nlp):
    review_body = reviews
    # review_id = row['review_id']
    # review_marketplace = row['marketplace']
    # customer_id = row['customer_id']
    # product_parent = row['product_parent']
    # product_title = row['product_title']
    # product_category = row['product_category']
    # date = str(row['review_date'])
    # star_rating = row['star_rating']
    # url = add_amazonlink(product_id)



    doc=nlp(review_body)
    #print("--- SPACY : Doc loaded ---")

    ## FIRST RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect
    ## RULE = M is child of A with a relationshio of amod
    rule1_pairs = []
    rule2_pairs = []
    rule3_pairs = []
    rule4_pairs = []
    rule5_pairs = []
    rule6_pairs = []
    rule7_pairs = []

    for token in doc:
        A = "999999"
        M = "999999"
        if token.dep_ == "amod" and not token.is_stop:
            M = token.text
            A = token.head.text

            # add adverbial modifier of adjective (e.g. 'most comfortable headphones')
            M_children = token.children
            for child_m in M_children:
                if(child_m.dep_ == "advmod"):
                    M_hash = child_m.text
                    M = M_hash + " " + M
                    break

            # negation in adjective, the "no" keyword is a 'det' of the noun (e.g. no interesting characters)
            A_children = token.head.children
            for child_a in A_children:
                if(child_a.dep_ == "det" and child_a.text == 'no'):
                    neg_prefix = 'not'
                    M = neg_prefix + " " + M
                    break

        if(A != "999999" and M != "999999"):
            if A in prod_pronouns :
                A = "product"
            dict1 = {"noun" : A, "adj" : M, "rule" : 1}
            rule1_pairs.append(dict1)

        # print("--- SPACY : Rule 1 Done ---")

        # # SECOND RULE OF DEPENDANCY PARSE -
        # # M - Sentiment modifier || A - Aspect
        # Direct Object - A is a child of something with relationship of nsubj, while
        # M is a child of the same something with relationship of dobj
        # Assumption - A verb will have only one NSUBJ and DOBJ
        children = token.children
        A = "999999"
        M = "999999"
        add_neg_pfx = False
        for child in children :
            if(child.dep_ == "nsubj" and not child.is_stop):
                A = child.text
                # check_spelling(child.text)

            if((child.dep_ == "dobj" and child.pos_ == "ADJ") and not child.is_stop):
                M = child.text
                #check_spelling(child.text)

            if(child.dep_ == "neg"):
                neg_prefix = child.text
                add_neg_pfx = True

        if (add_neg_pfx and M != "999999"):
            M = neg_prefix + " " + M

        if(A != "999999" and M != "999999"):
            if A in prod_pronouns :
                A = "product"
            dict2 = {"noun" : A, "adj" : M, "rule" : 2}
            rule2_pairs.append(dict2)




        # print("--- SPACY : Rule 2 Done ---")

        ## THIRD RULE OF DEPENDANCY PARSE -
        ## M - Sentiment modifier || A - Aspect
        ## Adjectival Complement - A is a child of something with relationship of nsubj, while
        ## M is a child of the same something with relationship of acomp
        ## Assumption - A verb will have only one NSUBJ and DOBJ
        ## "The sound of the speakers would be better. The sound of the speakers could be better" - handled using AUX dependency

        children = token.children
        A = "999999"
        M = "999999"
        add_neg_pfx = False
        for child in children :
            if(child.dep_ == "nsubj" and not child.is_stop):
                A = child.text
                # check_spelling(child.text)

            if(child.dep_ == "acomp" and not child.is_stop):
                M = child.text

            # example - 'this could have been better' -> (this, not better)
            if(child.dep_ == "aux" and child.tag_ == "MD"):
                neg_prefix = "not"
                add_neg_pfx = True

            if(child.dep_ == "neg"):
                neg_prefix = child.text
                add_neg_pfx = True

        if (add_neg_pfx and M != "999999"):
            M = neg_prefix + " " + M
                #check_spelling(child.text)

        if(A != "999999" and M != "999999"):
            if A in prod_pronouns :
                A = "product"
            dict3 = {"noun" : A, "adj" : M, "rule" : 3}
            rule3_pairs.append(dict3)
            #rule3_pairs.append((A, M, sid.polarity_scores(M)['compound'],3))
    # print("--- SPACY : Rule 3 Done ---")

        ## FOURTH RULE OF DEPENDANCY PARSE -
        ## M - Sentiment modifier || A - Aspect

        #Adverbial modifier to a passive verb - A is a child of something with relationship of nsubjpass, while
        # M is a child of the same something with relationship of advmod

        #Assumption - A verb will have only one NSUBJ and DOBJ

        children = token.children
        A = "999999"
        M = "999999"
        add_neg_pfx = False
        for child in children :
            if((child.dep_ == "nsubjpass" or child.dep_ == "nsubj") and not child.is_stop):
                A = child.text
                # check_spelling(child.text)

            if(child.dep_ == "advmod" and not child.is_stop):
                M = child.text
                M_children = child.children
                for child_m in M_children:
                    if(child_m.dep_ == "advmod"):
                        M_hash = child_m.text
                        M = M_hash + " " + child.text
                        break
                #check_spelling(child.text)

            if(child.dep_ == "neg"):
                neg_prefix = child.text
                add_neg_pfx = True

        if (add_neg_pfx and M != "999999"):
            M = neg_prefix + " " + M

        if(A != "999999" and M != "999999"):
            if A in prod_pronouns :
                A = "product"
            dict4 = {"noun" : A, "adj" : M, "rule" : 4}
            rule4_pairs.append(dict4)
            #rule4_pairs.append((A, M,sid.polarity_scores(M)['compound'],4)) # )

    # print("--- SPACY : Rule 4 Done ---")


        ## FIFTH RULE OF DEPENDANCY PARSE -
        ## M - Sentiment modifier || A - Aspect

        #Complement of a copular verb - A is a child of M with relationship of nsubj, while
        # M has a child with relationship of cop

        #Assumption - A verb will have only one NSUBJ and DOBJ

        children = token.children
        A = "999999"
        buf_var = "999999"
        for child in children :
            if(child.dep_ == "nsubj" and not child.is_stop):
                A = child.text
                # check_spelling(child.text)

            if(child.dep_ == "cop" and not child.is_stop):
                buf_var = child.text
                #check_spelling(child.text)

        if(A != "999999" and buf_var != "999999"):
            if A in prod_pronouns :
                A = "product"
            dict5 = {"noun" : A, "adj" : token.text, "rule" : 5}
            rule5_pairs.append(dict5)
            #rule5_pairs.append((A, token.text,sid.polarity_scores(token.text)['compound'],5))

    # print("--- SPACY : Rule 5 Done ---")

        ## SIXTH RULE OF DEPENDANCY PARSE -
        ## M - Sentiment modifier || A - Aspect
        ## Example - "It ok", "ok" is INTJ (interjections like bravo, great etc)

        children = token.children
        A = "999999"
        M = "999999"
        if(token.pos_ == "INTJ" and not token.is_stop):
            for child in children :
                if(child.dep_ == "nsubj" and not child.is_stop):
                    A = child.text
                    M = token.text
                    # check_spelling(child.text)

        if(A != "999999" and M != "999999"):
            if A in prod_pronouns :
                A = "product"
            dict6 = {"noun" : A, "adj" : M, "rule" : 6}
            rule6_pairs.append(dict6)

            #rule6_pairs.append((A, M,sid.polarity_scores(M)['compound'],6))

    # print("--- SPACY : Rule 6 Done ---")

    ## SEVENTH RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect
    ## ATTR - link between a verb like 'be/seem/appear' and its complement
    ## Example: 'this is garbage' -> (this, garbage)

        children = token.children
        A = "999999"
        M = "999999"
        add_neg_pfx = False
        for child in children :
            if(child.dep_ == "nsubj" and not child.is_stop):
                A = child.text
                # check_spelling(child.text)

            if((child.dep_ == "attr") and not child.is_stop):
                M = child.text
                #check_spelling(child.text)

            if(child.dep_ == "neg"):
                neg_prefix = child.text
                add_neg_pfx = True

        if (add_neg_pfx and M != "999999"):
            M = neg_prefix + " " + M

        if(A != "999999" and M != "999999"):
            if A in prod_pronouns :
                A = "product"
            dict7 = {"noun" : A, "adj" : M, "rule" : 7}
            rule7_pairs.append(dict7)
            #rule7_pairs.append((A, M,sid.polarity_scores(M)['compound'],7))



    #print("--- SPACY : Rules Done ---")


    aspects = []

    aspects = rule1_pairs + rule2_pairs + rule3_pairs +rule4_pairs +rule5_pairs + rule6_pairs + rule7_pairs

    # replace all instances of "it", "this" and "they" with "product"
    #aspects = [(A,M,P,r) if A not in prod_pronouns else ("product",M,P,r) for A,M,P,r in aspects ]

    # dic = {"review_id" : review_id , "aspect_pairs" : aspects, "review_marketplace" : review_marketplace
    # , "customer_id" : customer_id, "product_id" : product_id, "product_parent" : product_parent,
    # "product_title" : product_title, "product_category" : product_category, "date" : date, "star_rating" : star_rating, "url" : url}

    dic = {"aspect_pairs" : aspects}
    # dic = {"review_id" : review_id , "aspect_pairs" : aspects}
    return dic

In [42]:
for i, sentence in enumerate(sentences):
    print(i, sentence)

0 The food we had yesterday was delicious
1 My time in Italy was very enjoyable
2 I found the meal to be tasty
3 The internet was slow.
4 Our experience was suboptimal
5 anc work in the first 3 days but the proximity sensor on the left bud stopped working
6 shipped out immediately, received the item in good condition, no damage well packed, thank you shopee and thank you seller, will order again 👍😁👌🫰
7 Tysm for delivering my items safely. I only expected the seller gave some freebies, but he/she didn't 😂😂😾
8 Sobrang tagal lang dumating. Pero okay lang, sulit naman. Nakuha ko sya sa halagang 999 pesos. Nagsale kasi. 😍 Thank youuu! Legit !
9 been using this buds for 2 weeks na and I notice that sometimes it took so long to connect on my laptop and phone I don't know why. then sometimes nagkaproblem sound nya parang basag yung tunog
10 wrong color. poor packaging
11 Delivery and shipment were both okay.
12 If you intend to use this to attract your man, be prepared for a ravenous, passiona

In [37]:
for i, sentence in enumerate(sentences):
    print(i, apply_extraction(sentences[i], nlp))

0 {'aspect_pairs': [{'noun': 'food', 'adj': 'delicious', 'rule': 3}]}
1 {'aspect_pairs': [{'noun': 'time', 'adj': 'enjoyable', 'rule': 3}]}
2 {'aspect_pairs': [{'noun': 'meal', 'adj': 'tasty', 'rule': 3}]}
3 {'aspect_pairs': [{'noun': 'internet', 'adj': 'slow', 'rule': 3}]}
4 {'aspect_pairs': [{'noun': 'experience', 'adj': 'suboptimal', 'rule': 3}]}
5 {'aspect_pairs': [{'noun': 'bud', 'adj': 'left', 'rule': 1}]}
6 {'aspect_pairs': [{'noun': 'condition', 'adj': 'good', 'rule': 1}, {'noun': 'damage', 'adj': 'not well packed', 'rule': 1}]}
7 {'aspect_pairs': []}
8 {'aspect_pairs': []}
9 {'aspect_pairs': []}
10 {'aspect_pairs': [{'noun': 'color', 'adj': 'wrong', 'rule': 1}, {'noun': 'packaging', 'adj': 'poor', 'rule': 1}]}
11 {'aspect_pairs': [{'noun': 'Delivery', 'adj': 'okay', 'rule': 3}]}
12 {'aspect_pairs': [{'noun': 'result', 'adj': 'ravenous', 'rule': 1}, {'noun': 'result', 'adj': 'passionate', 'rule': 1}, {'noun': 'target', 'adj': 'intended', 'rule': 1}, {'noun': 'move', 'adj': '*',

---------------------------
END OF ASPECT GITHUB
---------------------------


In [38]:
aspects = []
for sentence in sentences:
  doc = nlp(sentence)
  descriptive_term = ''
  # target = ''
  target = []
  for token in doc:
    if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
      # target = token.text
      target.append(token.text)
    if token.pos_ == 'ADJ':
      prepend = ''
      for child in token.children:
        if child.pos_ != 'ADV':
          continue
        prepend += child.text + ' '
      descriptive_term = f"<{prepend}>" + token.text
  aspects.append({'aspect': target,
  # aspects.append({'aspect': target,
    'description': descriptive_term})

pprint(aspects)

[{'aspect': ['food'], 'description': '<>delicious'},
 {'aspect': ['time'], 'description': '<very >enjoyable'},
 {'aspect': ['meal'], 'description': '<>tasty'},
 {'aspect': ['internet'], 'description': '<>slow'},
 {'aspect': ['experience'], 'description': '<>suboptimal'},
 {'aspect': ['sensor'], 'description': '<>left'},
 {'aspect': ['damage'], 'description': '<>good'},
 {'aspect': ['seller'], 'description': ''},
 {'aspect': [], 'description': '<>Legit'},
 {'aspect': [], 'description': '<so >long'},
 {'aspect': [], 'description': '<>poor'},
 {'aspect': ['Delivery'], 'description': '<>okay'},
 {'aspect': ['stuff', 'man', 'dab'], 'description': '<>small'},
 {'aspect': ['effect', 'cats'], 'description': '<very >lovey'}]


Now our solution is starting to look more complete. We’re able to pick up aspects, even though our application doesn’t “know” anything beforehand. We haven’t hardcoded the aspects such as “food”, “time”, or “meal”. And we also haven’t hardcoded the adjectives such as “tasty”, “slow”, or “enjoyable”.

# Sentiment Analysis
Now that we successfully extracted the aspects and descriptions, it’s time to classify them as positive or negative. The goal here is to help the computer understand that tasty food is positive, while slow internet is negative. Computers don’t understand English, so we will need to try a few things before we have a working solution.

In [ ]:
from textblob import TextBlob
for aspect in aspects:
  aspect['sentiment'] = TextBlob(aspect['description']).sentiment
  
pprint(aspects)

[{'aspect': ['food'],
  'description': '<>delicious',
  'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)},
 {'aspect': ['time'],
  'description': '<very >enjoyable',
  'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)},
 {'aspect': ['meal'],
  'description': '<>tasty',
  'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)},
 {'aspect': ['internet'],
  'description': '<>slow',
  'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)},
 {'aspect': ['experience'],
  'description': '<>suboptimal',
  'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)},
 {'aspect': ['sensor'],
  'description': '<>left',
  'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)},
 {'aspect': ['damage'],
  'description': '<>good',
  'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)},
 {'aspect': ['seller'],
  'description': '',
  'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)},
 {'aspect': [],
  'description': '<>Legit',
  'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)},
 

In [ ]:
for i, sentence in enumerate(sentences):
    print(f'{i+1:02}). {sentence}')

01). The food we had yesterday was delicious
02). My time in Italy was very enjoyable
03). I found the meal to be tasty
04). The internet was slow.
05). Our experience was suboptimal
06). anc work in the first 3 days but the proximity sensor on the left bud stopped working
07). shipped out immediately, received the item in good condition, no damage well packed, thank you shopee and thank you seller, will order again 👍😁👌🫰
08). Tysm for delivering my items safely. I only expected the seller gave some freebies, but he/she didn't 😂😂😾
09). Sobrang tagal lang dumating. Pero okay lang, sulit naman. Nakuha ko sya sa halagang 999 pesos. Nagsale kasi. 😍 Thank youuu! Legit !
10). been using this buds for 2 weeks na and I notice that sometimes it took so long to connect on my laptop and phone I don't know why. then sometimes nagkaproblem sound nya parang basag yung tunog
11). wrong color. poor packaging
12). Delivery and shipment were both okay.
13). If you intend to use this to attract your man, 

In [ ]:
for i, aspect in enumerate(aspects):
    print(f'{i+1:02}.) "{sentences[i]}"')
    print(f'\tASPECT:"{aspect["aspect"]}",',
            f'DESCRIPTION:"{aspect["description"]}",',
            f'POLARITY:{"POSITIVE" if (aspect["sentiment"].polarity) > 0 else "NEGATIVE" if (aspect["sentiment"].polarity) < 0 else "NEUTRAL"}')
    print(f'\t"{aspect["sentiment"]}"\n')

01.) "The food we had yesterday was delicious"
	ASPECT:"['food']", DESCRIPTION:"<>delicious", POLARITY:NEUTRAL
	"Sentiment(polarity=0.0, subjectivity=0.0)"

02.) "My time in Italy was very enjoyable"
	ASPECT:"['time']", DESCRIPTION:"<very >enjoyable", POLARITY:NEUTRAL
	"Sentiment(polarity=0.0, subjectivity=0.0)"

03.) "I found the meal to be tasty"
	ASPECT:"['meal']", DESCRIPTION:"<>tasty", POLARITY:NEUTRAL
	"Sentiment(polarity=0.0, subjectivity=0.0)"

04.) "The internet was slow."
	ASPECT:"['internet']", DESCRIPTION:"<>slow", POLARITY:NEUTRAL
	"Sentiment(polarity=0.0, subjectivity=0.0)"

05.) "Our experience was suboptimal"
	ASPECT:"['experience']", DESCRIPTION:"<>suboptimal", POLARITY:NEUTRAL
	"Sentiment(polarity=0.0, subjectivity=0.0)"

06.) "anc work in the first 3 days but the proximity sensor on the left bud stopped working"
	ASPECT:"['sensor']", DESCRIPTION:"<>left", POLARITY:NEUTRAL
	"Sentiment(polarity=0.0, subjectivity=0.0)"

07.) "shipped out immediately, received the item i

# ASPECT EXTRACTION with CUSTOM SENTIMENT ANALYSIS NAIVE BAYES

In [ ]:
# Importing libraries
import pandas as pd
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pickle

model = pickle.load(open('modelNB.pkl', 'rb'))

# @app.route('/')
# def home():
#     return render_template('index.html')

def predict(raw_text: str):

    # Instantiate PorterStemmer
    p_stemmer = PorterStemmer()

    # Remove HTML
    review_text = BeautifulSoup(raw_text, features="lxml").get_text()

    # Remove non-letters
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)

    # Convert words to lower case and split each word up
    words = letters_only.lower().split()

    # Convert stopwords to a set
    stops = set(stopwords.words('english'))

    # Adding on stopwords that were appearing frequently in both positive and negative reviews
    stops.update(['app','shopee','shoppee','item','items','seller','sellers','bad'])

    # Remove stopwords
    meaningful_words = [w for w in words if w not in stops]

    # Stem words
    meaningful_words = [p_stemmer.stem(w) for w in meaningful_words]

    # Join words back into one string, with a space in between each word
    final_text = pd.Series(" ".join(meaningful_words))

    # Generate predictions
    pred = model.predict(final_text)[0]

    if pred == 1:
        output = "Negative"
    else:
        output = "Postive"

    return output, pred


text = """
anc work in the first 3 days but the proximity sensor on the left bud stopped working
"""
result = predict(text)
print(f'RAW_TEXT: \t{text}')
print(F'\tSENTIMENT: {result}')

RAW_TEXT: 	
anc work in the first 3 days but the proximity sensor on the left bud stopped working

	SENTIMENT: ('Negative', 1)


In [ ]:
for i, aspect in enumerate(aspects):
    print(f'{i+1:02}.) "{sentences[i]}"')
    print(f'\tASPECT:"{aspect["aspect"]}",',
            f'DESCRIPTION:"{aspect["description"]}",',
            f'SENTIMENT: {predict(aspect["description"])}'
    )

01.) "The food we had yesterday was delicious"
	ASPECT:"['food']", DESCRIPTION:"<>delicious", SENTIMENT: ('Postive', 0)
02.) "My time in Italy was very enjoyable"
	ASPECT:"['time']", DESCRIPTION:"<very >enjoyable", SENTIMENT: ('Postive', 0)
03.) "I found the meal to be tasty"
	ASPECT:"['meal']", DESCRIPTION:"<>tasty", SENTIMENT: ('Postive', 0)
04.) "The internet was slow."
	ASPECT:"['internet']", DESCRIPTION:"<>slow", SENTIMENT: ('Negative', 1)
05.) "Our experience was suboptimal"
	ASPECT:"['experience']", DESCRIPTION:"<>suboptimal", SENTIMENT: ('Postive', 0)
06.) "anc work in the first 3 days but the proximity sensor on the left bud stopped working"
	ASPECT:"['sensor']", DESCRIPTION:"<>left", SENTIMENT: ('Postive', 0)
07.) "shipped out immediately, received the item in good condition, no damage well packed, thank you shopee and thank you seller, will order again 👍😁👌🫰"
	ASPECT:"['damage']", DESCRIPTION:"<>good", SENTIMENT: ('Postive', 0)
08.) "Tysm for delivering my items safely. I onl